# Genetic Algorithm

In [14]:
import random as rndm
import time

## Part 1: Make Genes and Chromosomes

In [15]:
def check_initial(gene, initial):
    mapp = {}
    for i in range(9):
        mapp[gene[i]] = i
    for i in range(9):
        if initial[i] != 0 and gene[i] != initial[i]:
            temp = gene[i], gene[mapp[initial[i]]]
            gene[mapp[initial[i]]], gene[i] = temp
            mapp[initial[i]], mapp[temp[0]] = i, mapp[initial[i]]
    return gene 

def make_gene(initial=None):
    if initial is None:
        initial = [0] * 9
    gene = list(range(1, 10))
    rndm.shuffle(gene)
    gene = check_initial(gene, initial)
    return gene

In [16]:
def make_chromosome(initial=None):
    if initial is None:
        initial = [[0] * 9] * 9
    chromosome = []
    for i in range(9):
        chromosome.append(make_gene(initial[i]))
    return chromosome

## Part 2: Making First Generation

In [17]:
def make_population(count, initial=None):
    if initial is None:
        initial = [[0] * 9] * 9
    population = []
    for _ in range(count):
        population.append(make_chromosome(initial))
    return population

## Part 3: Fitness Function

In [18]:
def get_fitness(chromosome):
    """Calculate the fitness of a chromosome (puzzle)."""
    fitness = 0
    for i in range(9): # For each column
        seen = {}
        for j in range(9): # Check each cell in the column
            if chromosome[j][i] in seen:
                seen[chromosome[j][i]] += 1
            else:
                seen[chromosome[j][i]] = 1
        for key in seen: # Subtract fitness for repeated numbers
            fitness -= (seen[key] - 1)
    for m in range(3): # For each 3x3 square
        for n in range(3):
            seen = {}
            for i in range(3 * n, 3 * (n + 1)):  # Check cells in 3x3 square
                for j in range(3 * m, 3 * (m + 1)):
                    if chromosome[j][i] in seen:
                        seen[chromosome[j][i]] += 1
                    else:
                        seen[chromosome[j][i]] = 1
            for key in seen: # Subtract fitness for repeated numbers
                fitness -= (seen[key] - 1)
    return fitness

In [19]:
ch = make_chromosome()
print(get_fitness(ch))

def pch(ch):
    for i in range(9):
        for j in range(9):
            print(ch[i][j], end=" ")
        print("")


-55


## Part 4: Crossover and Mutation

### Crossover

In [20]:
def uniform_crossover(ch1, ch2):
    new_child_1 = []
    new_child_2 = []
    for i in range(9):
        x = rndm.randint(0, 1)
        if x == 1:
            new_child_1.append(ch1[i])
            new_child_2.append(ch2[i])
        elif x == 0:
            new_child_2.append(ch1[i])
            new_child_1.append(ch2[i])
    return new_child_1, new_child_2

def first_difference(g1, g2):
    for i in range(9):
        if g1[i] != g2[i]:
            return i
    return -1

def cycle_crossover(ch1, ch2):
    new_child_1 = []
    new_child_2 = [] 
    for i in range(9):
        mapp = {}
        for j in range(9):
            mapp[ch1[i][j]] = j
        k = first_difference(ch1[i], ch2[i])
        if k != -1:
            end = k
            gene_1 = ch1[i].copy()
            gene_2 = ch2[i].copy()
            while True:
                gene_1[k], gene_2[k] = gene_2[k], gene_1[k]
                k = mapp[gene_1[k]]     
                if k == end:
                    break  
            new_child_1.append(gene_1)
            new_child_2.append(gene_2)
        else:
            new_child_1.append(ch1[i])
            new_child_2.append(ch2[i])
    return new_child_1, new_child_2

### Mutation

In [21]:
def random_mutation(ch, pm, initial):
    for i in range(9):
        x = rndm.randint(0, 100)
        if x < pm * 100:
            ch[i] = make_gene(initial[i])
    return ch

def scramble_mutation(ch, pm, initial):
    for i in range(9):
        x = rndm.randint(0, 100)
        if x < pm * 100:
            start = rndm.randint(0, 7)
            end = rndm.randint(start + 1, 8)
            ch[i][start:end + 1] = rndm.sample(ch[i][start:end + 1], end - start + 1)
            ch[i] = check_initial(ch[i], initial[i])
    return ch

def insert_mutation(ch, pm, initial):
    for i in range(9):
        x = rndm.randint(0, 100)
        if x < pm * 100:
            idx = rndm.randint(0, 8)
            new_idx = rndm.randint(0, 8)
            while idx == new_idx:
                new_idx = rndm.randint(0, 8)
            temp = ch[i][idx]
            ch[i].pop(idx)
            ch[i].insert(new_idx, temp)
            ch[i] = check_initial(ch[i], initial[i])
    return ch

## Part 5: Implementing The Genetic Algorithm

Read the puzzle from the given address


In [22]:
def read_puzzle(address):
    puzzle = []
    f = open(address, 'r')
    for row in f:
        temp = row.split()
        puzzle.append([int(c) for c in temp])
    return puzzle

Get the mating pool using the random method

In [23]:
def r_get_mating_pool(population):
    fitness_list = []
    pool = []
    for chromosome in population:
        fitness = get_fitness(chromosome)
        fitness_list.append((fitness, chromosome))
    fitness_list.sort()
    weight = list(range(1, len(fitness_list) + 1))
    for _ in range(len(population)):
        ch = rndm.choices(fitness_list, weight)[0]
        pool.append(ch[1])
    return pool


Generate the offsprings from the mating pool

In [24]:
def get_offsprings(population, initial, pm, pc):
    new_pool = []
    i = 0
    while i < len(population):
        ch1 = population[i]
        ch2 = population[(i + 1) % len(population)]
        x = rndm.randint(0, 100)
        if x < pc * 100:
            algorithm = rndm.randint(0, 1)
            if algorithm == 0:
                ch1, ch2 = uniform_crossover(ch1, ch2)
            else:
                ch1, ch2 = cycle_crossover(ch1, ch2)
        algorithm = rndm.randint(0, 2)
        if algorithm == 0:
            new_pool.append(random_mutation(ch1, pm, initial))
            new_pool.append(random_mutation(ch2, pm, initial))
        elif algorithm == 1:
            new_pool.append(scramble_mutation(ch1, pm, initial))
            new_pool.append(scramble_mutation(ch2, pm, initial))
        else:
            new_pool.append(insert_mutation(ch1, pm, initial))
            new_pool.append(insert_mutation(ch2, pm, initial))
        i += 2
    return new_pool

In [25]:
# Population size
POPULATION = 1000

# Number of generations
REPETITION = 1000

# Probability of mutation
PM = 0.1

# Probability of crossover
PC = 0.95

# Main genetic algorithm function
def genetic_algorithm(initial_file):
    initial = read_puzzle(initial_file)
    population = make_population(POPULATION, initial)
    for _ in range(REPETITION):
        mating_pool = r_get_mating_pool(population)
        rndm.shuffle(mating_pool)
        population = get_offsprings(mating_pool, initial, PM, PC)
        fit = [get_fitness(c) for c in population]
        m = max(fit)
        if m == 0:
            return population
    return population

Run the algorithm and time it

In [26]:
tic = time.time()
r = genetic_algorithm("./sample_sudoku/Test2.txt")
toc = time.time()
print("time_taken: ", toc - tic)
fit = [get_fitness(c) for c in r]
m = max(fit)
print(max(fit))

# Print the chromosome with the highest fitness
for c in r:
    if get_fitness(c) == m:
        pch(c)
        break

KeyboardInterrupt: 